In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import sys
sys.path.append('../')

from utils.processing import *
from utils.visualisation import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, MinMaxScaler

np.random.seed(0)

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 80)

In [4]:
# load data set
df = pd.read_csv('../data/train.csv')
df_train, df_validate = train_test_split(df, test_size=0.30)

df_train.iloc[0]

listing_id                                                          224052
title                                         1 bed condo for sale in myra
address                                            9 meyappa chettiar road
property_name                                                         myra
property_type                                                        condo
tenure                                                            freehold
built_year                                                          2024.0
num_beds                                                               1.0
num_baths                                                              1.0
size_sqft                                                              474
floor_level                                                            NaN
furnishing                                                     unspecified
available_unit_types                                         1, 2, 3, 4 br
total_num_units          

In [5]:
def drop_cols_for_regression(df):
    drop_columns = [
        'address',
        'title',
        #'listing_id',
        'property_name',
        'total_num_units',
        'available_unit_types',
        'property_details_url',
        'elevation',
        'tenure',
        'property_type',
        'floor_level',
        'furnishing',
        'built_year',
        'planning_area',
        'subzone',
        'furnishing_partial',
        'furnishing_unfurnished',
        'furnishing_unspecified',
        'floor_level_ground',
        'floor_level_high',
        'floor_level_mid',
        'floor_level_low',
        'floor_level_penthouse',
        'floor_level_top',
        'line_cc',
        'line_ce',
        'line_cg',
        'line_dt',
        'line_ew',
        'line_ne',
        'line_ns',
        'line_te',
    ]
    """

        'gep_pri_sch_within_1km',
        'gep_pri_sch_within_1km_2km',
        'gep_pri_sch_outside_2km',
        'pri_sch_within_500m',
        'pri_sch_outside_500m',

        'furnishing_partial',
        'furnishing_unfurnished',
        'furnishing_unspecified',
    ]
    """
    return df.drop(columns=drop_columns)

def prepare_data_for_regression(df):
    adfs = read_aux_csv('../data')
    #merge_aux = dict((k, adfs[k]) for k in ['primary_schools', 'regions', 'mrt_stations'])
    merge_aux = dict((k, adfs[k]) for k in ['regions', 'mrt_stations'])
    df = join_aux(df, merge_aux)

    df = drop_cols_for_regression(df)

    return df

In [6]:
#df = prepare_data_for_regression(df)
#df.head()
#df_train[df_train.isna().any(axis=1)]

df_train = preprocess(df_train)
df_train = prepare_data_for_regression(df_train)
df_train.head()

,listing_id,num_beds,num_baths,size_sqft,lat,lng,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,region,nearest_mrt_distance_in_km
0,224052,1.0,1.0,474,1.330709,103.868391,1060900.0,1,0,1,0,c,0.131235
1,247665,2.0,2.0,915,1.360919,103.892050,1029900.0,1,0,0,1,ne,0.780849
2,130438,4.0,4.0,4717,1.300620,103.892783,8059800.0,1,0,1,0,c,0.973647
3,223406,3.0,1.0,699,1.316655,103.805632,554400.0,0,1,0,1,c,0.212855
4,606217,3.0,3.0,1249,1.315795,103.832096,3738000.0,1,0,1,0,c,0.760161


In [7]:
X_train = df_train.drop('price', axis=1)
y_train = df_train['price']

In [8]:
def data_pipeline(model=None, polynomial=2):
    preprocessor = make_column_transformer(
                    (StandardScaler(), ['num_beds', 'num_baths', 'size_sqft']),
                    (OneHotEncoder(handle_unknown='ignore'), ["region"]),
                    remainder='passthrough')
    
    pipeline = [preprocessor]
    
    if polynomial > 0:
        pipeline.append(PolynomialFeatures(polynomial))
    
    if model != None:
        pipeline.append(model)
        
    return make_pipeline(*pipeline)

def pipelined_model(model):
    return TransformedTargetRegressor(regressor=data_pipeline(model), transformer=StandardScaler())

def transform_data(df):
    p = data_pipeline(polynomial=0)
    p.fit(df)
    
    return pd.DataFrame(data=p.transform(df), columns=p.get_feature_names_out())

In [9]:
transform_data(X_train).head()

,standardscaler__num_beds,standardscaler__num_baths,standardscaler__size_sqft,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,remainder__listing_id,remainder__lat,remainder__lng,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__nearest_mrt_distance_in_km
0,-1.675193,-1.131695,-0.695263,1.0,0.0,0.0,0.0,0.0,224052.0,1.330709,103.868391,1.0,0.0,1.0,0.0,0.131235
1,-0.878157,-0.432616,-0.436065,0.0,0.0,0.0,1.0,0.0,247665.0,1.360919,103.892050,1.0,0.0,0.0,1.0,0.780849
2,0.715915,0.965541,1.798560,1.0,0.0,0.0,0.0,0.0,130438.0,1.300620,103.892783,1.0,0.0,1.0,0.0,0.973647
3,-0.081121,-1.131695,-0.563019,1.0,0.0,0.0,0.0,0.0,223406.0,1.316655,103.805632,0.0,1.0,0.0,1.0,0.212855
4,-0.081121,0.266462,-0.239757,1.0,0.0,0.0,0.0,0.0,606217.0,1.315795,103.832096,1.0,0.0,1.0,0.0,0.760161


In [10]:
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    score = np.sqrt(mse)
    return score
    
def run_models(models, X, y):
    def rmse_scorer():
        return make_scorer(rmse)
    
    model_scores = dict()
    
    for model in models:
        regr = pipelined_model(model)
        scores = cross_validate(regr, X, y, cv=10, scoring=rmse_scorer())
        regr.fit(X, y)
        model_scores[model.__class__.__name__] = [regr, scores]
        print(model.__class__.__name__, scores['test_score'].mean())
        
    return model_scores

In [11]:
models = [
            #KNeighborsRegressor(),
            #Lasso(),
            #Ridge(),
            #DecisionTreeRegressor(),
            #GradientBoostingRegressor(),
            #AdaBoostRegressor(),
            #RandomForestRegressor(max_depth=10),
            xgb.XGBRegressor(),
            #xgb.XGBRegressor(n_estimators=100, min_child_weight=10, max_depth=5, learning_rate=0.7),
            #xgb.XGBRegressor(n_estimators=200),
            #xgb.XGBRegressor(max_depth=8, learning_rate=0.3, min_child_weight=1),
            #xgb.XGBRegressor(max_depth=8, learning_rate=0.3, min_child_weight=1, n_estimators=200)
        ]

model_scores = run_models(models, X_train, y_train)

XGBRegressor 788017.4417684453


In [12]:
"""
import math
best_score = math.inf
best_model = None

for k, v in model_scores.items():
    regr, scores = v
    score = scores['test_score'].mean()
    
    print(k, score)
    
    if score < best_score:
        best_score = score
        best_model = regr
        
        
parameters = {'regressor__xgbregressor__max_depth':[1, 6, 8],
              #'regressor__xgbregressor__n_estimators':[100, 150, 200],
              'regressor__xgbregressor__min_child_weight': [1, 5, 10],
              'regressor__xgbregressor__learning_rate': [0.1, 0.3, 0.7],
             }

m = pipelined_model(xgb.XGBRegressor())
model = GridSearchCV(m, parameters)


model.fit(X_train, y_train)

best_params = model.best_params_
print(best_params)
"""

"\nimport math\nbest_score = math.inf\nbest_model = None\n\nfor k, v in model_scores.items():\n    regr, scores = v\n    score = scores['test_score'].mean()\n    \n    print(k, score)\n    \n    if score < best_score:\n        best_score = score\n        best_model = regr\n        \n        \nparameters = {'regressor__xgbregressor__max_depth':[1, 6, 8],\n              #'regressor__xgbregressor__n_estimators':[100, 150, 200],\n              'regressor__xgbregressor__min_child_weight': [1, 5, 10],\n              'regressor__xgbregressor__learning_rate': [0.1, 0.3, 0.7],\n             }\n\nm = pipelined_model(xgb.XGBRegressor())\nmodel = GridSearchCV(m, parameters)\n\n\nmodel.fit(X_train, y_train)\n\nbest_params = model.best_params_\nprint(best_params)\n"

In [13]:
y_validate = df_validate['price']
X_validate = prepare_data_for_regression(preprocess(df_validate.drop('price', axis=1), is_target=True, num_beds=df_train['num_beds'].median(), num_baths=df_train['num_baths'].median()))

transform_data(X_validate).head()

,standardscaler__num_beds,standardscaler__num_baths,standardscaler__size_sqft,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__region_nan,remainder__listing_id,remainder__lat,remainder__lng,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__nearest_mrt_distance_in_km
0,0.681451,0.930558,0.009133,0.0,0.0,0.0,1.0,0.0,0.0,297997.0,1.357815,103.881064,1.0,0.0,1.0,0.0,0.516576
1,-0.866943,-1.102396,-0.070694,1.0,0.0,0.0,0.0,0.0,0.0,907902.0,1.310429,103.802821,1.0,0.0,1.0,0.0,0.741763
2,-0.092746,-0.424745,-0.051139,0.0,0.0,1.0,0.0,0.0,0.0,235585.0,1.446546,103.803916,0.0,1.0,0.0,1.0,0.737947
3,2.229846,0.930558,0.065515,1.0,0.0,0.0,0.0,0.0,0.0,284232.0,1.327671,103.841705,1.0,0.0,1.0,0.0,0.837827
4,-0.092746,-0.424745,-0.051191,0.0,0.0,1.0,0.0,0.0,0.0,746242.0,1.424190,103.844932,0.0,1.0,0.0,1.0,1.248562


In [14]:
y_validate = df_validate['price']
X_validate = prepare_data_for_regression(preprocess(df_validate.drop('price', axis=1), is_target=True, num_beds=df_train['num_beds'].median(), num_baths=df_train['num_baths'].median()))

for k, v in model_scores.items():
    regr, scores = v
    predict_validate = regr.predict(X_validate)
    
    print(k, rmse(y_validate, predict_validate))

XGBRegressor 2968614.6711450815


In [15]:
# Retrain using best model
fmodel = pipelined_model(xgb.XGBRegressor())

df_ftrain = pd.read_csv('../data/train.csv')
df_ftrain = preprocess(df_ftrain)
df_ftrain = prepare_data_for_regression(df_ftrain)
df_ftrain.head()

X_ftrain = df_ftrain.drop('price', axis=1)
y_ftrain = df_ftrain['price']

fmodel.fit(X_ftrain, y_ftrain)
predict_ftrain = fmodel.predict(X_ftrain)
print(rmse(y_ftrain, predict_ftrain))

280968.9692425118


In [16]:
df_test = preprocess(pd.read_csv('../data/test.csv'), is_target=True, num_beds=df_train['num_beds'].median(), num_baths=df_train['num_baths'].median())

# further preprocessing
X_test = prepare_data_for_regression(df_test)

y_predict = fmodel.predict(X_test)

X_test['Predicted'] = y_predict

In [17]:
y_predict

array([1146131.2, 1552100.4, 1223100.9, ..., 3422706.5,  610468.8,
       4046057.8], dtype=float32)

In [18]:
submission = X_test[['Predicted']]
submission.to_csv('submission3.csv', index=True, index_label='id', header=True, columns=['Predicted'])